In [ ]:
# # Would Play with Logistic Regression (adopted from my hw3)

# import gzip
# from collections import defaultdict
# import random

# # Load data with functions from stub
# def readGz(path):
#     for l in gzip.open(path, 'rt'):
#         yield eval(l)

# def readJSON(path):
#     for l in gzip.open(path, 'rt'):
#         d = eval(l)
#         u = d['userID']
#         try:
#             g = d['gameID']
#         except Exception as e:  # In case the 'gameID' attribute is missing.
#             g = None
#         yield u, g, d

# allHours = []
# for l in readJSON("train.json.gz"):
#     allHours.append(l)

# # Split the data into training and validation sets.
# hoursTrain = allHours[:165000]
# hoursValid = allHours[165000:]

# # Data structures for the would-play baseline (from baseline.py)
# gameCount = defaultdict(int)  # Dictionary to keep track of how many times each game was played.
# totalPlayed = 0  # Total number of games played.

# # Find out which games a user hasn't played yet.
# userPlayedGames = defaultdict(set)

# # Populate gameCount dictionary and totalPlayed count.
# for user, game, _ in readJSON("train.json.gz"):
#     gameCount[game] += 1
#     totalPlayed += 1
#     userPlayedGames[user].add(game)
    
# # Sort games by popularity (number of times played).
# mostPopular = [(gameCount[x], x) for x in gameCount]
# mostPopular.sort()
# mostPopular.reverse()

# gamePopularity = {}
# for g in gameCount.keys():
#     idx = mostPopular.index((gameCount[g],g))
#     gamePopularity[g] = 1.0 - idx/len(mostPopular)

# # Create validation set which includes both positive and negative samples.
# allGames = list(gameCount.keys())  # All distinct games in our dataset.
# validationSet = []

# # Generate negative samples for the valid set.
# for user, game, _ in hoursValid:
#     negativeGame = random.choice(allGames)
#     while negativeGame in userPlayedGames[user]:
#         negativeGame = random.choice(allGames)
#     validationSet.append((user, game, 1))        # Positive example: The actual game played by the user
#     validationSet.append((user, negativeGame, 0)) # Negative example: A randomly chosen game (not played by the user)

# # Create a dictionary with games as keys and sets of users who played them as values.
# gameUsers = defaultdict(set)
# for user, game, _ in hoursTrain:
#     gameUsers[game].add(user)

# # Calculate Jaccard similarity between two sets.
# def jaccard_similarity(set1, set2):
#     intersection = len(set1 & set2)
#     union = len(set1 | set2)
#     return intersection / union if union != 0 else 0

# # Precompute Jaccard similarities for all game pairs.
# game_pair_similarities = defaultdict(dict)
# all_games = list(gameUsers.keys())

# for i, game1 in enumerate(all_games):
#     for j, game2 in enumerate(all_games):
#         if j <= i:
#             continue  # No need to compute similarity twice for the same pair
#         similarity = jaccard_similarity(gameUsers[game1], gameUsers[game2])
#         game_pair_similarities[game1][game2] = similarity
#         game_pair_similarities[game2][game1] = similarity

# from sklearn.linear_model import LogisticRegression

# # Prepare the features and labels.
# X = []
# y = []

# for user, game, actual in validationSet:
#     # Feature 1: Jaccard similarity
#     max_similarity = 0
#     for game_prime in userPlayedGames[user]:
#         if game_prime in game_pair_similarities[game]:
#             similarity = game_pair_similarities[game][game_prime]
#             max_similarity = max(max_similarity, similarity)
#     # Feature 2: Popularity
#     popularity = gamePopularity[game]
#     X.append([max_similarity, popularity])
#     y.append(actual)

# # Train a logistic regression model.
# clf = LogisticRegression()
# clf.fit(X, y)

# # Predict on the validation set.
# y_pred = clf.predict(X)

# # Open the test file and prediction file.
# predictions = open("predictions_Played.csv", 'w')

# for l in open("pairs_Played.csv"):
#     if l.startswith("userID"):
#         # Write the header to the output file.
#         predictions.write(l)
#         continue
#     u, g = l.strip().split(',')
    
#     # Extract features for the given (u, g) pair.
#     # Feature 1: Jaccard similarity.
#     max_similarity = 0
#     if u in userPlayedGames:  # Make sure the user exists in the training data.
#         for game_prime in userPlayedGames[u]:
#             if game_prime in game_pair_similarities[game]:
#                 similarity = game_pair_similarities[game][game_prime]
#                 max_similarity = max(max_similarity, similarity)
    
#     # Feature 2: Popularity.
#     popularity = gamePopularity[g]
    
#     # Make a prediction using the logistic regression model.
#     pred = clf.predict([[max_similarity, popularity]])[0]  # clf is the trained logistic regression model.
    
#     # Write the prediction to the output file.
#     predictions.write(u + ',' + g + ',' + str(pred) + '\n')

# # Close the prediction file.
# predictions.close()

In [ ]:
# # Would Play with CatBoost (idea came from office hour)
# import gzip
# from collections import defaultdict
# import random
# import pandas as pd

# # Load data with functions from stub
# def readGz(path):
#     for l in gzip.open(path, 'rt', encoding = "utf-8"):
#         yield eval(l)

# def readJSON(path):
#     for l in gzip.open(path, 'rt', encoding = "utf-8"):
#         d = eval(l)
#         u = d['userID']
#         try:
#             g = d['gameID']
#         except Exception as e:  # In case the 'gameID' attribute is missing.
#             g = None
#         yield u, g, d

# allHours = []
# for l in readJSON("train.json.gz"):
#     allHours.append(l)

# # Split the data into training and validation sets.
# hoursTrain = allHours[:165000]
# hoursValid = allHours[165000:]

# # Data structures for the would-play baseline (from baseline.py)
# gameCount = defaultdict(int)  # Dictionary to keep track of how many times each game was played.
# totalPlayed = 0  # Total number of games played.

# # Find out which games a user hasn't played yet.
# userPlayedGames = defaultdict(set)

# # Populate gameCount dictionary and totalPlayed count.
# for user, game, _ in readJSON("train.json.gz"):
#     gameCount[game] += 1
#     totalPlayed += 1
#     userPlayedGames[user].add(game)
    
# # Sort games by popularity (number of times played).
# mostPopular = [(gameCount[x], x) for x in gameCount]
# mostPopular.sort()
# mostPopular.reverse()

# # gamePopularity = {}
# # for g in gameCount.keys():
# #     idx = mostPopular.index((gameCount[g],g))
# #     gamePopularity[g] = 1.0 - idx/len(mostPopular)

# # Create validation set which includes both positive and negative samples.
# allGames = list(gameCount.keys())  # All distinct games in our dataset.
# trainSet = []
# validSet = []

# # Generate negative samples for the valid set.
# for user, game, _ in hoursTrain:
#     negativeGame = random.choice(allGames)
#     while negativeGame in userPlayedGames[user]:
#         negativeGame = random.choice(allGames)
#     trainSet.append((user, game, 1))        # Positive example: The actual game played by the user
#     trainSet.append((user, negativeGame, 0)) # Negative example: A randomly chosen game (not played by the user)

# # Generate negative samples for the valid set.
# for user, game, _ in hoursValid:
#     negativeGame = random.choice(allGames)
#     while negativeGame in userPlayedGames[user]:
#         negativeGame = random.choice(allGames)
#     validSet.append((user, game, 1))        # Positive example: The actual game played by the user
#     validSet.append((user, negativeGame, 0)) # Negative example: A randomly chosen game (not played by the user)

# # Create a dictionary with games as keys and sets of users who played them as values.
# gameUsers = defaultdict(set)
# for user, game, _ in hoursTrain:
#     gameUsers[game].add(user)

# # Calculate Jaccard similarity between two sets.
# def jaccard_similarity(set1, set2):
#     intersection = len(set1 & set2)
#     union = len(set1 | set2)
#     return intersection / union if union != 0 else 0

# # Precompute Jaccard similarities for all game pairs.
# game_pair_similarities = defaultdict(dict)
# all_games = list(gameUsers.keys())

# for i, game1 in enumerate(all_games):
#     for j, game2 in enumerate(all_games):
#         if j <= i:
#             continue  # No need to compute similarity twice for the same pair
#         similarity = jaccard_similarity(gameUsers[game1], gameUsers[game2])
#         game_pair_similarities[game1][game2] = similarity
#         game_pair_similarities[game2][game1] = similarity

# # Prepare the features and labels.
# X_train = []
# y_train = []

# for user, game, actual in trainSet:
#     # Feature 1: Jaccard similarity
#     similarity = 0
#     count = 0
#     for game_prime in userPlayedGames[user]:
#         if game_prime in game_pair_similarities[game]:
#             similarity += game_pair_similarities[game][game_prime]
#             count += 1
#     if count == 0:
#         similarity = 0
#     else:
#         similarity /= count
#     # Feature 2: Popularity
#     # popularity = gamePopularity[game]
#     popularity = gameCount[game]
#     X_train.append([similarity, popularity])
#     # X_train.append([similarity])
#     y_train.append(actual)
    
# X_train = pd.DataFrame(X_train, columns=['Similarity','Popularity'])
# # X_train = pd.DataFrame(X_train, columns=['Similarity'])
# y_train = pd.DataFrame(y_train, columns=['Label'])

# X_valid = []
# y_valid = []
    
# for user, game, actual in validSet:
#     # Feature 1: Jaccard similarity
#     similarity = 0
#     count = 0
#     for game_prime in userPlayedGames[user]:
#         if game_prime in game_pair_similarities[game]:
#             similarity += game_pair_similarities[game][game_prime]
#             count += 1
#     if count == 0:
#         similarity = 0
#     else:
#         similarity /= count
#     # Feature 2: Popularity
#     # popularity = gamePopularity[game]
#     popularity = gameCount[game]
#     X_valid.append([similarity, popularity])
#     # X_valid.append([similarity])
#     y_valid.append(actual)

# X_valid = pd.DataFrame(X_valid, columns=['Similarity','Popularity'])
# # X_valid = pd.DataFrame(X_valid, columns=['Similarity'])
# y_valid = pd.DataFrame(y_valid, columns=['Label'])

# from catboost import CatBoostClassifier, Pool, metrics, cv
# from sklearn.metrics import accuracy_score

# model = CatBoostClassifier(learning_rate = 0.01)

# model.fit(
#     X_train, y_train,
# #    cat_features=categorical_features_indices,
#     eval_set=(X_valid, y_valid),
# #     logging_level='Verbose',  # you can uncomment this for text output
# #    plot=True
# );

# # Open the test file and prediction file.
# predictions = open("predictions_Played.csv", 'w')

# for l in open("pairs_Played.csv"):
#     if l.startswith("userID"):
#         # Write the header to the output file.
#         predictions.write(l)
#         continue
#     u, g = l.strip().split(',')
    
#     # Extract features for the given (u, g) pair.
#     # Feature 1: Jaccard similarity.
#     similarity = 0
#     count = 0
#     if u in userPlayedGames:  # Make sure the user exists in the training data.
#         for game_prime in userPlayedGames[u]:
#             if game_prime in game_pair_similarities[game]:
#                 similarity += game_pair_similarities[game][game_prime]
#                 count += 1
#         similarity /= count
#     # Feature 2: Popularity.
#     # popularity = gamePopularity[g]
#     popularity = gameCount[game]
#     # Make a prediction using the logistic regression model.
#     pred = model.predict([[similarity, popularity]])[0]  # clf is the trained logistic regression model.
#     # pred = model.predict([[similarity]])[0]
    
#     # Write the prediction to the output file.
#     predictions.write(u + ',' + g + ',' + str(pred) + '\n')

# # Close the prediction file.
# predictions.close()

In [ ]:
# # Would Play with XGBoost (idea came from office hour) using optuna for best parameters

# import gzip
# from collections import defaultdict
# import random
# import pandas as pd

# # Load data with functions from stub
# def readGz(path):
#     for l in gzip.open(path, 'rt', encoding = "utf-8"):
#         yield eval(l)

# def readJSON(path):
#     for l in gzip.open(path, 'rt', encoding = "utf-8"):
#         d = eval(l)
#         u = d['userID']
#         try:
#             g = d['gameID']
#         except Exception as e:  # In case the 'gameID' attribute is missing.
#             g = None
#         yield u, g, d

# allHours = []
# for l in readJSON("train.json.gz"):
#     allHours.append(l)

# # Split the data into training and validation sets.
# hoursTrain = allHours[:165000]
# hoursValid = allHours[165000:]

# # Data structures for the would-play baseline (from baseline.py)
# gameCount = defaultdict(int)  # Dictionary to keep track of how many times each game was played.
# totalPlayed = 0  # Total number of games played.

# # Find out which games a user hasn't played yet.
# userPlayedGames = defaultdict(set)

# # Populate gameCount dictionary and totalPlayed count.
# for user, game, _ in readJSON("train.json.gz"):
#     gameCount[game] += 1
#     totalPlayed += 1
#     userPlayedGames[user].add(game)
    
# # Sort games by popularity (number of times played).
# mostPopular = [(gameCount[x], x) for x in gameCount]
# mostPopular.sort()
# mostPopular.reverse()

# # gamePopularity = {}
# # for g in gameCount.keys():
# #     idx = mostPopular.index((gameCount[g],g))
# #     gamePopularity[g] = 1.0 - idx/len(mostPopular)

# # Create validation set which includes both positive and negative samples.
# allGames = list(gameCount.keys())  # All distinct games in our dataset.
# trainSet = []
# validSet = []

# # Generate negative samples for the valid set.
# for user, game, _ in hoursTrain:
#     negativeGame = random.choice(allGames)
#     while negativeGame in userPlayedGames[user]:
#         negativeGame = random.choice(allGames)
#     trainSet.append((user, game, 1))        # Positive example: The actual game played by the user
#     trainSet.append((user, negativeGame, 0)) # Negative example: A randomly chosen game (not played by the user)

# # Generate negative samples for the valid set.
# for user, game, _ in hoursValid:
#     negativeGame = random.choice(allGames)
#     while negativeGame in userPlayedGames[user]:
#         negativeGame = random.choice(allGames)
#     validSet.append((user, game, 1))        # Positive example: The actual game played by the user
#     validSet.append((user, negativeGame, 0)) # Negative example: A randomly chosen game (not played by the user)

# # Create a dictionary with games as keys and sets of users who played them as values.
# gameUsers = defaultdict(set)
# for user, game, _ in hoursTrain:
#     gameUsers[game].add(user)

# # Calculate Jaccard similarity between two sets.
# def jaccard_similarity(set1, set2):
#     intersection = len(set1 & set2)
#     union = len(set1 | set2)
#     return intersection / union if union != 0 else 0

# # Precompute Jaccard similarities for all game pairs.
# game_pair_similarities = defaultdict(dict)
# all_games = list(gameUsers.keys())

# for i, game1 in enumerate(all_games):
#     for j, game2 in enumerate(all_games):
#         if j <= i:
#             continue  # No need to compute similarity twice for the same pair
#         similarity = jaccard_similarity(gameUsers[game1], gameUsers[game2])
#         game_pair_similarities[game1][game2] = similarity
#         game_pair_similarities[game2][game1] = similarity

# # Prepare the features and labels.
# X_train = []
# y_train = []

# for user, game, actual in trainSet:
#     # Feature 1: Jaccard similarity
#     similarity = 0
#     count = 0
#     for game_prime in userPlayedGames[user]:
#         if game_prime in game_pair_similarities[game]:
#             similarity += game_pair_similarities[game][game_prime]
#             count += 1
#     if count != 0:
#         similarity /= count
#     # Feature 2: Popularity
#     # popularity = gamePopularity[game] 
#     # popularity = gameCount[game]
#     # X_train.append([similarity, popularity])
#     X_train.append([similarity])
#     y_train.append(actual)

# X_valid = []
# y_valid = []
    
# for user, game, actual in validSet:
#     # Feature 1: Jaccard similarity
#     similarity = 0
#     count = 0
#     for game_prime in userPlayedGames[user]:
#         if game_prime in game_pair_similarities[game]:
#             similarity += game_pair_similarities[game][game_prime]
#             count += 1
#     if count != 0:
#         similarity /= count
#     # Feature 2: Popularity
#     # popularity = gamePopularity[game]
#     # popularity = gameCount[game]
#     # X_valid.append([similarity, popularity])
#     X_valid.append([similarity])
#     y_valid.append(actual)

# import xgboost as xgb

# import optuna

# # 1. Define an objective function to be maximized.
# def objective(trial):
    
#     dtrain = xgb.DMatrix(X_train, y_train)
#     # 2. Suggest values of the hyperparameters using a trial object.
#     param = {
#         'silent': 1,
#         'objective': 'binary:logistic',
#         'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
#         'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
#         'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)
#     }

#     bst = xgb.train(param, dtrain)
#     y_predict = model.predict(X_valid)
#     accuracy = sum([1 if i==j else 0 for i, j in zip (y_valid, y_predict)])/ len(y_valid)
#     # print(f"The current model accuracy on test set is {accuracy}")
#     return accuracy

# # 3. Create a study object and optimize the objective function.
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1000)

# # Open the test file and prediction file.
# predictions = open("predictions_Played.csv", 'w')

# for l in open("pairs_Played.csv"):
#     if l.startswith("userID"):
#         # Write the header to the output file.
#         predictions.write(l)
#         continue
#     u, g = l.strip().split(',')
    
#     # Extract features for the given (u, g) pair.
#     # Feature 1: Jaccard similarity.
#     similarity = 0
#     count = 0
#     if u in userPlayedGames:  # Make sure the user exists in the training data.
#         for game_prime in userPlayedGames[u]:
#             if game_prime in game_pair_similarities[game]:
#                 similarity += game_pair_similarities[game][game_prime]
#                 count += 1
#         similarity /= count
#     # Feature 2: Popularity.
#     # popularity = gamePopularity[g]
#     # Spopularity = gameCount[g]
    
#     # Make a prediction using the logistic regression model.
#     # pred = model.predict([[similarity, popularity]])[0]  # clf is the trained logistic regression model.
#     pred = model.predict([similarity])[0]
    
#     # Write the prediction to the output file.
#     predictions.write(u + ',' + g + ',' + str(pred) + '\n')

# # Close the prediction file.
# predictions.close()

In [4]:
# Would Play with further feature engineering (idea from professor during the lecture)

import gzip
from collections import defaultdict
import random

# Function to read data from a gzipped JSON file line by line
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u, g, d

# Reading and storing the dataset
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

# Splitting dataset into training and validation sets
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

# Initializing data structures to store various information
userPlayedGamesAll = defaultdict(set)
allGamesCount = defaultdict(int)

# Calculating frequencies and adding games and users to sets
for user, game, _ in allHours:
    userPlayedGamesAll[user].add(game)
    allGamesCount[game] += 1

# Preparing training and validation sets
userPlayedGamesTrain = defaultdict(set)
userPlayedGamesValid = defaultdict(set)

# Populating the training set
for user, game, _ in hoursTrain:
    userPlayedGamesTrain[user].add(game)

# Generating negative samples for the validation set
hoursValidNegative = []
for user, game, _ in hoursValid:
    userPlayedGamesValid[user].add(game)
    negativeGame = random.choice(list(allGamesCount.keys() - userPlayedGamesAll[user]))
    hoursValidNegative.append((user, negativeGame, 0))
    userPlayedGamesValid[user].add(negativeGame)

# Reading test data
userPlayedGamesTest = defaultdict(set)
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        continue
    u, g = l.strip().split(',')
    userPlayedGamesTest[u].add(g)

# Jaccard similarity function
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom if denom > 0 else 0

# UserGamePredictor class with combined prediction strategies
class UserGamePredictor():
    def __init__(self, jaccard_threshold=0.025):
        self.jaccard_threshold = jaccard_threshold
    
    def predict(self, user, game, gamesPerUser=None):
        if gamesPerUser is None: gamesPerUser = userPlayedGamesTest[user]

        # Frequency-based prediction
        popularHalfGames = sorted(gamesPerUser, key=lambda x: allGamesCount[x], reverse=True)[:len(gamesPerUser) // 2]
        freq_based_prediction = game in popularHalfGames

        # Jaccard similarity-based prediction
        maxSim = 0
        usersWhoPlayedGame = set(userPlayedGamesAll[game])
        for g2 in userPlayedGamesAll[user]:
            sim = Jaccard(usersWhoPlayedGame, userPlayedGamesAll[g2])
            if sim > maxSim:
                maxSim = sim

        jaccard_based_prediction = maxSim > self.jaccard_threshold and len(userPlayedGamesAll[game]) > 8

        # Combining both strategies
        return int(freq_based_prediction or jaccard_based_prediction)

    # Validation method
    def validate(self):
        accuracy = 0
        # Evaluate accuracy for positive samples
        for user, game, _ in hoursValid:
            accuracy += (self.predict(user, game, userPlayedGamesValid[user]) == 1)
        # Evaluate accuracy for negative samples
        for user, game, _ in hoursValidNegative:
            accuracy += (self.predict(user, game, userPlayedGamesValid[user]) == 0)
        # Calculate total accuracy
        accuracy /= (len(hoursValid)+len(hoursValidNegative))
        return accuracy

# Instantiate the model and validate
model = UserGamePredictor()
accuracy = model.validate()
print(f"The accuracy of my model on validation set is {accuracy}")

# Writing predictions to a file
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):  # Write header line
        predictions.write(l)
        continue
    u, g = l.strip().split(',')  # Split each line into userID and gameID
    pred = model.predict(u, g)  # Make a prediction
    predictions.write(u + ',' + g + ',' + str(pred) + '\n')  # Write the prediction

predictions.close()  # Close the file

The accuracy of my model on validation set is 0.7316231623162316


In [17]:
# Time Play (adopted from my hw3)

from tqdm.notebook import tqdm
import numpy as np

# Calculate global average
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) / len(trainHours)

# Initialize dictionaries
hoursPerUserItem = defaultdict(float)
gamesPerUser = defaultdict(int)
usersPerItem = defaultdict(int)
betaU = defaultdict(float)
betaI = defaultdict(float)

# Populate the dictionaries
for user, game, d in hoursTrain:
    hour = d['hours_transformed']
    if user not in gamesPerUser:
        gamesPerUser[user] = set()
    if game not in usersPerItem:
        usersPerItem[game] = set()
    hoursPerUserItem[(user,game)] = hour
    gamesPerUser[user].add(game)
    usersPerItem[game].add(user)

alpha = globalAverage  # Initialize alpha

def iterate(lamb):
    newAlpha = 0
    newBetaU = {}
    newBetaI = {}
    
    for key, value in hoursPerUserItem.items():
        user, game = key
        hour = value
        newAlpha += (hour-betaU[user]-betaI[game])
    newAlpha /= len(hoursPerUserItem.items())
    
    for u in betaU.keys():
        newU = 0.0
        for g in gamesPerUser[u]:
            hour = hoursPerUserItem[(u,g)]
            newU += (hour - newAlpha - betaI[g])
        newU /= (lamb + len(gamesPerUser[u]))    
        newBetaU[u] = newU

    for i in betaI.keys():
        newI = 0.0
        for u in usersPerItem[i]:
            hour = hoursPerUserItem[(u,i)]
            newI += (hour - newAlpha - betaU[u])
        newI /= (lamb + len(usersPerItem[i]))    
        newBetaI[i] = newI
            
    return newAlpha, newBetaU, newBetaI

# Initialize variables to store the best lambda and its corresponding MSE
best_lambda = None
best_mse = float('inf')

# Iterate over a range of lambda values
for lamb in np.arange(4.9445,4.9455,0.0001):  # Adjust this range and values as necessary
    
    print(f'-------------lambda={lamb}-------------')
    # Initialize beta values for this lambda
    for u in gamesPerUser.keys():
        betaU[u] = 0

    for g in usersPerItem.keys():
        betaI[g] = 0

    alpha = globalAverage  # Reset alpha

    # Perform the iterations
    for i in tqdm(range(100)):
        alpha, betaU, betaI = iterate(lamb)
        current_mse = 0.0
        valid_mse = 0.0
        for user, game, d in hoursValid:
            hour = d['hours_transformed']
            prediction = alpha + betaU.get(user,0)+ betaI.get(game,0)
            current_mse += ((hour-prediction)**2)/len(hoursValid)
        if (abs(valid_mse-current_mse)<1e-9):
            break
        valid_mse = current_mse

    # Calculate MSE for this lambda
    
    print(f"Lambda: {lamb}, MSE: {valid_mse:.10f}")

    # Update best lambda and MSE if current MSE is lower
    if valid_mse < best_mse:
        best_mse = valid_mse
        best_lambda = lamb

# Print the best lambda and its MSE
print(f"Best lambda: {best_lambda}, with MSE: {best_mse:.10f}")

-------------lambda=4.9445-------------


  0%|          | 0/100 [00:00<?, ?it/s]

Lambda: 4.9445, MSE: 2.9906277299
-------------lambda=4.944599999999999-------------


  0%|          | 0/100 [00:00<?, ?it/s]

Lambda: 4.944599999999999, MSE: 2.9906277297
-------------lambda=4.944699999999999-------------


...

Lambda: 4.945299999999998, MSE: 2.9906277288
-------------lambda=4.945399999999998-------------


  0%|          | 0/100 [00:00<?, ?it/s]

Lambda: 4.945399999999998, MSE: 2.9906277288
-------------lambda=4.945499999999997-------------


  0%|          | 0/100 [00:00<?, ?it/s]

Lambda: 4.945499999999997, MSE: 2.9906277287
Best lambda: 4.945499999999997, with MSE: 2.9906277287


In [19]:
# Continue the time play to make prediction with best lambda
for i in tqdm(range(100)):
    alpha, betaU, betaI = iterate(4.95)
    current_mse = 0.0
    valid_mse = 0.0
    for user, game, d in hoursValid:
        hour = d['hours_transformed']
        prediction = alpha + betaU.get(user,0)+ betaI.get(game,0)
        current_mse += ((hour-prediction)**2)/len(hoursValid)
    if (abs(valid_mse-current_mse)<1e-7):
        break
    valid_mse = current_mse
    
# Open the test file and prediction file.        
predictions = open("predictions_Hours.csv", 'w')

for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        # Write the header to the output file.
        predictions.write(l)
        continue
    u, g = l.strip().split(',')
        
    bu, bi = betaU[u], betaI[g]
    # Write the prediction to the output file.
    predictions.write(u + ',' + g + ',' + str(alpha+bu+bi) + '\n')

# Close the prediction file.
predictions.close()

  0%|          | 0/100 [00:00<?, ?it/s]